In [5]:
from python_speech_features import mfcc
import scipy
import scipy.io.wavfile as wav
import numpy as np
import librosa
import pandas as pd

from tempfile import TemporaryFile
import os
import pickle
import random
import operator

import math

In [37]:
def distance (inst1, inst2) :
    dist = 0
    # mm1 = inst1[0]
    # cm1 = inst1[1]
    # mm2 = inst2[0]
    # cm2 = inst2[1]
    # dist = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    # dist += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    # dist += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    # dist -= k
    x = inst1[0]
    y = inst2[0]
    dist = np.linalg.norm(x-y)
    # dist = np.dot(inst1[0],inst2[0])/(np.linalg.norm(inst1[0])*np.linalg.norm(inst2[0]))
    return dist

In [33]:
def get_neighbors (x_test, inst, k) :
    dist = []
    for x in range(len(x_test)) :
        val = distance(x_test[x], inst)
        dist.append((x_test[x][1], val))
    dist.sort(key=lambda x: x[1])
    neighbors_list = []
    for x in range(k) :
        neighbors_list.append(dist[x][0])
    return neighbors_list

In [8]:
def nearest_neighbors(neighbors_list) :
    class_count = {}
    
    for x in range(len(neighbors_list)) :
        val = neighbors_list[x]
        if val in class_count :
            class_count[val] += 1
        else :
            class_count[val] = 1
        
    sorter = sorted(class_count.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [9]:
def model_accuracy(y_test, y_predict) :
    correct = 0
    for x in range(len(y_test)) :
        if y_test[x][-1] == y_predict[x] :
            correct += 1
    
    return 1.0 * correct / len(y_test)

In [10]:
def feature_extractor(file) :
    audio, sample_rate = librosa.load(file)
    mfcc_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfcc_scaled_features = np.mean(mfcc_features.T, axis=0)
    
    return mfcc_scaled_features

In [16]:
dataset_path = "./archive/Data/genres_original/"
dataset_file = open("extracted_data.dat", 'wb')
i = 0

for folder in os.listdir(dataset_path) :
    i += 1
    if i == 11 :
        break
    for file in os.listdir(dataset_path + folder) :
        try :
            # (rate, sig) = wav.read(dataset_path + folder + "/" + file)
            # mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            # covariance = np.cov(np.matrix.transpose(mfcc_feat))
            # mean_matrix = mfcc_feat.mean(0)
            # feature = (mean_matrix, covariance, i)
            file_name = dataset_path + folder + "/" + file
            # print(file_name)
            feature = (feature_extractor(file_name), i)
            pickle.dump(feature, dataset_file)
        except Exception as e :
            print("exception: ",e,"in folder: ", folder, "in file: ", file)

dataset_file.close() 

C:\Users\Lenovo\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


exception:   in folder:  jazz in file:  jazz.00054.wav


In [73]:
dataset = []
def load_dataset (filename, split, train_set, test_set) :
    with open("extracted_data.dat", 'rb') as file :
        while True :
            try :
                dataset.append(pickle.load(file))
            except EOFError :
                file.close()
                break
            
    for x in dataset :
        if random.random() < split :
            train_set.append(x)
        else :
            test_set.append(x)



train_set = []
test_set = []
load_dataset("extracted_data.dat", 0.66, train_set, test_set)
# print(test_set)
feature_df = pd.DataFrame(dataset, columns=['features','class'])
feature_df.head(10)

,features,class
0,"[-113.59882, 121.57067, -19.162262, 42.36394, ...",1
1,"[-207.52383, 123.98514, 8.947019, 35.86715, 2....",1
2,"[-90.757164, 140.44087, -29.084547, 31.686691,...",1
3,"[-199.57513, 150.0861, 5.663404, 26.855282, 1....",1
4,"[-160.35417, 126.20948, -35.581394, 22.139256,...",1
5,"[-177.80176, 118.20316, -17.57063, 30.760931, ...",1
6,"[-190.06882, 130.27023, -36.362114, 33.036217,...",1
7,"[-179.3723, 136.44833, -26.662588, 39.96545, 5...",1
8,"[-121.38979, 122.49833, -14.734729, 46.122135,...",1
9,"[-213.26424, 115.16476, -11.69817, 39.015358, ...",1


In [75]:
length = len(test_set)
predict = []
for x in range(len(test_set)) :
    predict.append(nearest_neighbors(get_neighbors(test_set, test_set[x], 5)))
# print(predict)
# print(len(test_set))
accuracy = model_accuracy(test_set, predict)
test_set_class = pd.DataFrame(test_set, columns = ['features','class'])
test_set_class = test_set_class['class']

from sklearn.metrics import confusion_matrix
con_met = confusion_matrix(predict, test_set_class)
print(con_met)
print(accuracy)

[[27  0  1  0  0  0  1  0  0  0]
 [ 0 20  0  0  0  2  0  0  0  0]
 [ 5  2 29  2  3  2  0  2  3  1]
 [ 0  0  0 12  0  1  1  0  0  1]
 [ 0  0  0  2 22  0  0  0  2  1]
 [ 2  3  5  2  1 31  1  3  4  1]
 [ 2  0  0  1  0  0 25  0  0  1]
 [ 0  0  0  2  2  0  0 23  1  0]
 [ 0  1  2  3  1  2  0  1 21  1]
 [ 1  0  4  5  0  1  2  2  2 36]]
0.7299703264094956


In [40]:
from collections import defaultdict
results = defaultdict(int)

directory = "./archive/Data/genres_original"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [70]:
test_file = "./metal.00000.wav"
test_file_feature = feature_extractor(test_file)
print(test_file_feature)
test_pred = nearest_neighbors(get_neighbors(dataset, test_file_feature, 100))
# test_pred = nearest_neighbors(get_neighbors(dataset, feature, 5))
print(results[test_pred])

[-52.940296    50.09013    -11.491269    46.324932     6.5430007
  10.116378     5.4855285    8.844898     5.9299426    6.625154
  -4.0330343    9.778293    -3.1172316    3.653597    -6.502195
   1.0854731   -4.482196     4.709674    -2.755327     0.4985645
  -9.29338      2.0262656   -3.4876702    2.5047514   -4.658039
  -1.8967493   -5.6836166   -3.1856866   -4.818261     0.85869867
  -5.738822    -0.4105425   -5.2758536    0.46732435  -0.6027184
  -0.65776074  -2.4302607    1.9490243   -4.047967    -2.8445647 ]
pop
